In [65]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
%matplotlib inline

In [66]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [67]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

In [68]:
train_images = (train_images - 127.5) / 127.5

In [69]:
BATCH_SIZE = 256
BUFFER_SIZE = 60000

In [70]:
datasets = tf.data.Dataset.from_tensor_slices(train_images)

In [71]:
datasets = datasets.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [72]:
# 创建生成器
def generator_model():
    model = tf.keras.Sequential()
    # 生成器一般不用偏执 use_bias False
    model.add(tf.keras.layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(512, use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(28*28*1, use_bias=False, activation="tanh"))
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.Reshape((28, 28, 1)))
   
    return model

In [73]:
# 创建辨别器
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(256, use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU()) 
    
    # 这种写法和上面的效果相同
    # model.add(tf.keras.layers.Activation(tf.nn.leaky_relu))
    
    model.add(tf.keras.layers.Dense(1))
    
    return model

In [74]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [75]:
def discriminator_loss(real_out, fake_out):
    real_loss = cross_entropy(tf.ones_like(real_out), real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out), fake_out)
    return real_loss + fake_loss

In [76]:
def generator_loss(fake_out):
    return cross_entropy(tf.ones_like(fake_out), fake_out)

In [77]:
generator_opt = tf.keras.optimizers.Adam(1e-4) 
discriminator_opt = tf.keras.optimizers.Adam(1e-4) 

In [78]:
EPOCHS = 100
noise_dim = 100
num_exp_to_generator =  16
seed = tf.random.normal([num_exp_to_generator, noise_dim])

In [79]:
generator = generator_model()
discriminator = discriminator_model()

In [80]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        real_out = discriminator(images)
        gen_images = generator(noise)
        fake_out = discriminator(gen_images)
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    gradient_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_opt.apply_gradients(zip(gradient_gen, generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(gradient_disc, discriminator.trainable_variables))

In [81]:
def generator_plot_images(model, test_noise):
    pre_images = model(test_noise)
    fig = plt.figure(figsize=(4, 4))
    for i in range(pre_images.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((np.squeeze(pre_images[i]) + 1)/2, cmap="gray")
        plt.axis("off")

In [82]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        generator_plot_images(generator, seed)

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
            print('.', end="")
        generator_plot_images(generator, seed)